In [40]:
!pip install torchsummary

In [41]:
#notes added to make sense of Dr. A.'s code.

#Code credit: Aladdin Persson (https://www.youtube.com/watch?v=IHq1t7NxS8k)
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
from torchsummary import summary

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv= nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias=False), #This is a same convolution. The input height and width are the same after the convolution.
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias=False), 
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
            return self.conv(x)
            
class UNET(nn.Module):
    def __init__(
            self, in_channels=1, out_channels=1, features=[32, 64, 128, 256, 512], #added 32 feature layer per Dr. A.'s code
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #recall that this is what roughly halves the size of the image each time.

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))
        
        # bottleneck part of UNET
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        
        
        #final convolution part of unet
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        
        #while we are progressing forward through the unet, we are also collecting skip connections, as part of 
        #the architecture
        
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [43]:
x = torch.randn((3, 1, 161, 161)) #3 objects, 1 channel, 161 x 161 pixels
x = x.to('cuda')
summary(model, (256, 256), 1, 'cuda')

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 1, 3, 3], but got 3-dimensional input of size [2, 256, 256] instead

In [ ]:
model = UNET(in_channels=1, out_channels=1)
model = model.to('cuda')
 

In [ ]:
x2 = torch.randn(64, 1, 512, 512)

In [ ]:
preds = model(x)
print(preds.shape)
print(x.shape)

print(x2.shape)
preds2 = model(x2)
print(preds2.shape)